In [ ]:
import torch.optim as optim
import gloss_proc
from torch import nn
from gloss_model import GlossModel
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter


In [ ]:
# get data for training
gp = gloss_proc.GlossProcess.load_checkpoint()
# gp.save_dataset()
# gdata,label=gp.load_dataset()
data, label = gp.get_all_gdata()
input_size = data.shape[2]
class_no = len(label)
input_size, class_no


In [ ]:
# provide input and class size
model = GlossModel(input_size,class_no)
optim = optim.Adam(model.parameters(), lr=0.1)
loss_fn= nn.CrossEntropyLoss()
model

In [ ]:
# split x , y for training and set epoch



In [ ]:
# initialize sumamry writer
writer=SummaryWriter()

In [ ]:
# Start model training
model.train()

for i in range(epoch):
    optim.zero_grad()
    out = model(x_train)
    loss = loss_fn(out, y_train.argmax())
    loss.backward()
    optim.step()
    writer.add_scalar("Loss/epoch", loss.item(), i)

writer.close()


In [ ]:
# save the model
torch.save(model, "swaram_lstm.pt")
